In [1]:
import os
from tqdm import tqdm
from PIL import Image

def scan_file_path_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            lst.append(os.path.join(root, file))
    return lst


def scan_file_name_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            # remove the extension
            file_name = os.path.splitext(file)[0]
            lst.append(file_name)
    return lst


audio_text = r"Data\rawData\Audio_text"

In [2]:
path_lst = scan_file_path_in_folder("Data\\PreporcessData\\Audio_key_text\\L01_V001")
name_lst = scan_file_name_in_folder(audio_text)
print(name_lst)

['L01_V001', 'L01_V002', 'L01_V003', 'L01_V004', 'L01_V005', 'L01_V006', 'L01_V007', 'L01_V008', 'L01_V009', 'L01_V010', 'L01_V011', 'L01_V012', 'L01_V013', 'L01_V014', 'L01_V015', 'L01_V016', 'L01_V017', 'L01_V018', 'L01_V019', 'L01_V020', 'L01_V021', 'L01_V022', 'L01_V023', 'L01_V024', 'L01_V025', 'L01_V026', 'L01_V027', 'L01_V028', 'L01_V029', 'L01_V030', 'L01_V031', 'L02_V001', 'L02_V002', 'L02_V003', 'L02_V004', 'L02_V005', 'L02_V006', 'L02_V007', 'L02_V008', 'L02_V009', 'L02_V010', 'L02_V011', 'L02_V012', 'L02_V013', 'L02_V014', 'L02_V015', 'L02_V016', 'L02_V017', 'L02_V018', 'L02_V019', 'L02_V020', 'L02_V021', 'L02_V022', 'L02_V023', 'L02_V024', 'L02_V025', 'L02_V026', 'L02_V027', 'L02_V028', 'L02_V029', 'L02_V030', 'L02_V031', 'L03_V001', 'L03_V002', 'L03_V003', 'L03_V004', 'L03_V005', 'L03_V006', 'L03_V007', 'L03_V008', 'L03_V009', 'L03_V010', 'L03_V011', 'L03_V012', 'L03_V013', 'L03_V014', 'L03_V015', 'L03_V016', 'L03_V017', 'L03_V018', 'L03_V019', 'L03_V020', 'L03_V021', 'L0

In [3]:
import timm
import torch

beitv2 = timm.create_model(
    "beitv2_base_patch16_224.in1k_ft_in22k",
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
).to("cuda")
beitv2.eval()
for param in beitv2.parameters():
    param.requires_grad = False
beitv2_data_config = timm.data.resolve_model_data_config(beitv2)
beitv2_transofms = timm.data.create_transform(**beitv2_data_config, is_training=False)

In [4]:
for i in tqdm(range(1, len(name_lst), 2)):
    name = name_lst[i]
    path_lst = scan_file_path_in_folder(
        f"Data\\rawData\\Keyframes\\Keyframes_{name[0:3]}\\keyframes\\{name}"
    )
    sub_name_lst = scan_file_name_in_folder(
        f"Data\\rawData\\Keyframes\\Keyframes_{name[0:3]}\\keyframes\\{name}"
    )
    for path, sub_name in zip(path_lst, sub_name_lst):
        out_path = f"Data\\PreporcessData\\BEITv2\\{name}_{sub_name}.pt"
        # check if the file is already exist
        if os.path.exists(out_path):
            continue
        # open the image
        image = Image.open(path).convert("RGB")
        image = image.resize((800, 800))

        with torch.no_grad():
            output = beitv2.forward_features(
                beitv2_transofms(image).unsqueeze(0).to("cuda")
            )
            torch.save(output, out_path)

100%|██████████| 363/363 [47:05<00:00,  7.78s/it]
